In [1]:
# import common packages
import numpy as np

import qiskit
from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import Operator, QuantumInstance

# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

In [3]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order
map_type = 'parity'

h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -8.854072040283643
# of electrons: 4
# of spin orbitals: 12


In [4]:
# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule.num_orbitals for x in remove_list]
freeze_list = [x % molecule.num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule.num_orbitals for x in freeze_list]

# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
qubit_reduction = True if map_type == 'parity' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = qubitOp.two_qubit_reduced_operator(num_particles) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

print(qubitOp.print_operators())
print(qubitOp)

IIII	(-0.20765933501970615+0j)
IIIZ	(-0.09376337484626751+0j)
IIZX	(-0.0031775814549315218+0j)
IIIX	(0.0031775814549315218+0j)
IIXX	(-0.001251396599930934+0j)
IIYY	(0.001251396599930934+0j)
IIZZ	(-0.21162509515109124+0j)
IIXZ	(0.019200533863090865+0j)
IIXI	(0.019200533863090865+0j)
IIZI	(0.358102699457708+0j)
IZII	(0.09376337484626757+0j)
ZXII	(0.0031775814549315244+0j)
IXII	(0.0031775814549315244+0j)
XXII	(-0.0012513965999309228+0j)
YYII	(0.0012513965999309228+0j)
ZZII	(-0.21162509515109132+0j)
XZII	(-0.01920053386309085+0j)
XIII	(0.01920053386309085+0j)
ZIII	(-0.35810269945770806+0j)
IZIZ	(-0.12182774215818912+0j)
IZZX	(0.012144897228116725+0j)
IZIX	(-0.012144897228116725+0j)
IZXX	(0.03169874598732462+0j)
IZYY	(-0.03169874598732462+0j)
IXIZ	(0.012144897228116725+0j)
ZXIZ	(0.012144897228116725+0j)
IXZX	(-0.0032659954996823014+0j)
ZXZX	(-0.0032659954996823014+0j)
IXIX	(0.0032659954996823014+0j)
ZXIX	(0.0032659954996823014+0j)
IXXX	(-0.008650156860638998+0j)
ZXXX	(-0.008650156860638998+

In [5]:
# setup HartreeFock state
HF_state = HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type, 
                       qubit_reduction)

# setup UCCSD variational form
var_form = UCCSD(qubitOp.num_qubits, depth=1, 
                   num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                   active_occupied=[0], active_unoccupied=[0, 1],
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=qubit_reduction, num_time_slices=1)

In [6]:
parameters = [0.9999999] * 8
circuit = var_form.construct_circuit(parameters, use_basis_gates=False)

In [7]:
circuit.draw(output='text', line_length=350)

In [8]:
backend_sim = Aer.get_backend('unitary_simulator')

In [9]:
result = qiskit.execute(circuit, backend_sim).result()

In [10]:
unitary = result.get_unitary()
print(unitary.shape)
print(unitary)

(16, 16)
[[ 0.11647636+6.36313165e-02j -0.18140114-9.90998820e-02j
   0.        +0.00000000e+00j  0.13841994+7.56191499e-02j
  -0.18140114-9.90998820e-02j  0.28251547+1.54338888e-01j
   0.        +0.00000000e+00j -0.21557624-1.17769822e-01j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j
  -0.33574003-1.83415591e-01j  0.522884  +2.85652795e-01j
   0.        -1.09992952e-15j -0.39899183-2.17970204e-01j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.25618976+1.39957089e-01j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j
  -0.39899183-2.17970204e-01j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j  0.        +1.16999601e-15j
  -0.73846024-4.03422617e-01j  0.        +0.00000000e+00j]
 [ 0.21557624+1.17769822e-01j  0.13841994+7.56191499e-02j
   